# Сучасні технології програмування. ЛР1
### Іванійчук А., КА-22мп

## Технічні моменти

In [1]:
import sys
import time
from datetime import datetime
import hashlib
import logging
import json

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s|%(lineno)s| %(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout
)
logging = logging.getLogger(__name__)

In [12]:
from IPython.core.display import HTML
HTML("""
<style>
div.output_stderr {
    background: #C4FFC0;
}
</style>
""")

## Реалізація основних класів

In [4]:
class BlockHeader:
    def __init__(
        self, 
        prevBlockHash: str, 
        txs: str, 
        bits: str = "ffff001f",  
        nonce: int = 0,
        version: int = 1,
        timestamp: int = int(time.time())
    ):
        self.version = version
        self.prevBlockHash = prevBlockHash
        self.merkleRoot = self.__gen_merkle_root(txs)
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = nonce
        self.blockHash = None
    
    @staticmethod
    def __sha256_encode(text: str, rounds: int = 2):
        """
        Perform SHA256 encryption
        """
        res = text[0] if isinstance(text, tuple) else text
        for _ in range(rounds):
            res = hashlib.sha256(res.encode("UTF-8")).hexdigest()
            
        return res
    
    def __gen_merkle_root(self, txs: str):
        """
        This is techical function which generates some kind 
        of imitation of the real merkle root
        """
        return self.__sha256_encode(txs, rounds=1)
    
    def mine(self):
        """
        Perform block mining
        """
        val = "".join([
            str(self.version),
            self.prevBlockHash,
            self.merkleRoot,
            str(self.timestamp)
        ])
        while True:
            attempt = self.__sha256_encode("".join([val, self.bits,
                                                    str(self.nonce)]))            
            if attempt[:4] == "0000":
                self.blockHash = attempt
                break
            self.nonce += 1
    
    def info(self):
        """
        Return info about block header
        """
        return {
            "version": self.version,
            "prevBlockHash": self.prevBlockHash,
            "merkleRoot": self.merkleRoot,
            "timestamp": self.timestamp,
            "bits": self.bits,
            "nonce": self.nonce,
            "blockHash": self.blockHash,
        }

In [5]:
class Block:
    def __init__(
        self,
        height: int = 0,
        prevBlockHash: str = "0" * 64,
    ):
        self.height: int = height
        self.blockSize: int = 1,
        self.txs: str = f"<user1> sent {height} coins to <user2>",
        self.blockHeader: BlockHeader = self.__get_block_header(
            prevBlockHash, self.txs),
        self.txCount: int = 1,
    
    def __get_block_header(self, prevBlockHash, txs):
        """
        This method creates header for block object, which includes 
        block hash, so it actually runs mining proccess
        """
        logging.log(20, "mining...")
        header = BlockHeader(prevBlockHash, txs)
        header.mine()
        logging.log(20, "success")
        return header
        
    def info(self):
        """
        Return info about block
        """
        return {
            "height": self.height,
            "blockSize": self.blockSize[0] if isinstance(
                self.blockSize, tuple) else self.blockSize,
            "blockHeader": self.blockHeader[0].info() if isinstance(
                self.blockHeader, tuple) else self.blockHeader.info(),
            "txCount": self.txCount[0] if isinstance(self.txCount, tuple) 
            else self.txCount,
            "txs": self.txs[0] if isinstance(self.txs, tuple) else self.txs
        }    

In [10]:
class Blockchain:
    def __init__(self):
        self.chain = []
    
    def start_chain(self):
        """
        This function actually inits the blockchain 
        by appending genesis block to it
        """
        logging.log(20, "starting blockchain...")
        self.chain.append(Block())
        logging.log(20, "genesis block generated successfully")
    
    def __get_previous_block(self):
        """
        Get the last block from the chain to generate the new one 
        """
        return self.chain[-1]
    
    def new_block(self):
        """
        Add new block to the blockchain
        """
        logging.log(20, "adding new block...")
        block_height = len(self.chain)
        prev_block_header = self.__get_previous_block().blockHeader
        prev_block_header = prev_block_header[0] if isinstance(
            prev_block_header, tuple) else prev_block_header
        new_block = Block(block_height, prev_block_header.blockHash)
        self.chain.append(new_block)
        logging.log(20, f"new block added successfully; \
                    chain lenght: {len(self.chain)}")
        
    def info(self):
        """
        Return info about the blockchain
        """
        return json.dumps([block.info() for block in self.chain], indent=2)

## Тестовий приклад

In [11]:
chain = Blockchain()
chain.start_chain()
chain.new_block()
print(chain.info())

INFO|10| 2023-01-20 05:25:23 - starting blockchain...
INFO|19| 2023-01-20 05:25:23 - mining...
INFO|22| 2023-01-20 05:25:24 - success
INFO|12| 2023-01-20 05:25:24 - genesis block generated successfully
INFO|24| 2023-01-20 05:25:24 - adding new block...
INFO|19| 2023-01-20 05:25:24 - mining...
INFO|22| 2023-01-20 05:25:24 - success
INFO|31| 2023-01-20 05:25:24 - new block added successfully; chain lenght: 2
[
  {
    "height": 0,
    "blockSize": 1,
    "blockHeader": {
      "version": 1,
      "prevBlockHash": "0000000000000000000000000000000000000000000000000000000000000000",
      "merkleRoot": "6b1d11354810742557e1503f7bf8199c9d24080dfbd9c841e1082d6b0ec5e9b1",
      "timestamp": 1674185085,
      "bits": "ffff001f",
      "nonce": 75436,
      "blockHash": "0000b5182e9a9ac11cb79ce56e65cbcf6630efddb5be57faa1d3e899f92948a7"
    },
    "txCount": 1,
    "txs": "<user1> sent 0 coins to <user2>"
  },
  {
    "height": 1,
    "blockSize": 1,
    "blockHeader": {
      "version": 1,
     